# Stretch Goal Approaches

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
from models import weather_classifier, ConvLSTM_predictor
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from DataGenerators import WeatherImageGenerator, SequenceWeatherImageGenerator
import matplotlib.pyplot as plt
import numpy as np

### Load Data

In [ ]:
x_data = np.load("images2.npy")
y_data = np.load("labels2.npy")

x_data = np.moveaxis(x_data, 1, -1)
x_data = x_data / 255 # Normalize images.

label_encoder = LabelEncoder() 
y_data = label_encoder.fit_transform(y_data) # Encode images as numbers instead of strings.
y_data = to_categorical(y_data) # Create an one-hot-encoded vector for each label.

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10)

# Train Weather Classifier

In [48]:
num_classes = y_data.shape[1]
input_shape = x_data.shape[1:]
batch_size = 32
epochs = 30

train_generator = WeatherImageGenerator(X_train, y_train, batch_size=batch_size)

model = weather_classifier(num_classes=num_classes, input_shape=input_shape)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 156, 246, 64)      32064     
_________________________________________________________________
batch_normalization_10 (Batc (None, 156, 246, 64)      256       
_________________________________________________________________
activation_23 (Activation)   (None, 156, 246, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 74, 119, 32)       204832    
_________________________________________________________________
batch_normalization_11 (Batc (None, 74, 119, 32)       128       
_________________________________________________________________
activation_24 (Activation)   (None, 74, 119, 32)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 33, 55, 32)        102432    
__________

In [ ]:
model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test,y_test))

# Train Weather Image Generation Model

In [49]:
sequence_size = 4
input_shape = (sequence_size,) + x_data[1].shape
batch_size = 6
epochs = 30

train_generator = SequenceWeatherImageGenerator(x_data, batch_size=batch_size, sequence_size=sequence_size)

model = ConvLSTM_predictor2(input_shape=input_shape)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_48 (ConvLSTM2D) (None, 4, 311, 491, 30)   420120    
_________________________________________________________________
activation_27 (Activation)   (None, 4, 311, 491, 30)   0         
_________________________________________________________________
conv_lst_m2d_49 (ConvLSTM2D) (None, 307, 487, 1)       3104      
_________________________________________________________________
activation_28 (Activation)   (None, 307, 487, 1)       0         
Total params: 423,224
Trainable params: 423,224
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit_generator(train_generator, epochs=epochs, callbacks=[checkpoint])